In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.oauth2 import service_account
from apiclient.discovery import build

In [3]:
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import json # JSON encoder and decoder for Python
import requests # Library for sending HTTP requests

In [5]:
import gapandas as gp

In [6]:
from gapandas import connect, query
service = connect.get_service('/Users/wangxiang/Desktop/ga-with-vic-596a1cbf5744.json')

In [7]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '/Users/wangxiang/Desktop/ga-with-vic-596a1cbf5744.json'
VIEW_ID = '111816020'

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

def get_report(analytics):
  """Queries the Analytics Reporting API V4.

  Args:
    analytics: An authorized Analytics Reporting API V4 service object.
  Returns:
    The Analytics Reporting API V4 response.
  """
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'dimensions': [{'name': 'ga:country'}]
        }]
      }
  ).execute()

def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response.

  Args:
    response: An Analytics Reporting API V4 response.
  """
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])

      for header, dimension in zip(dimensionHeaders, dimensions):
        print (header + ': ' + dimension)

      for i, values in enumerate(dateRangeValues):
        print ('Date range: ' + str(i))
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          print (metricHeader.get('name') + ': ' + value)

def main():
  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  print_response(response)

if __name__ == '__main__':
  main()

ga:country: (not set)
Date range: 0
ga:sessions: 25
ga:country: Australia
Date range: 0
ga:sessions: 62
ga:country: Austria
Date range: 0
ga:sessions: 2
ga:country: Brazil
Date range: 0
ga:sessions: 4
ga:country: Cambodia
Date range: 0
ga:sessions: 2
ga:country: Canada
Date range: 0
ga:sessions: 36
ga:country: China
Date range: 0
ga:sessions: 150
ga:country: Finland
Date range: 0
ga:sessions: 4
ga:country: France
Date range: 0
ga:sessions: 4
ga:country: Germany
Date range: 0
ga:sessions: 6
ga:country: Hong Kong
Date range: 0
ga:sessions: 104
ga:country: India
Date range: 0
ga:sessions: 2
ga:country: Indonesia
Date range: 0
ga:sessions: 12
ga:country: Ireland
Date range: 0
ga:sessions: 22
ga:country: Israel
Date range: 0
ga:sessions: 1
ga:country: Italy
Date range: 0
ga:sessions: 1
ga:country: Japan
Date range: 0
ga:sessions: 62
ga:country: Macao
Date range: 0
ga:sessions: 7
ga:country: Malaysia
Date range: 0
ga:sessions: 163
ga:country: Mexico
Date range: 0
ga:sessions: 4
ga:country: M

# 撈PV資料

In [12]:
payload = {
    'start_date': '2021-07-05',
    'end_date': '2021-07-11',
    'metrics': 'ga:pageviews',
    'dimensions': 'ga:pagePath'
}
df = query.run_query(service, '111816020', payload)

In [48]:
df.head()

,pagePath,pageviews
0,(not set),0
1,/,95455
2,/2017_stockholderevent_plus_0608,1
3,/2019_stockholderevent_SHARP,24
4,/56公分抽屜式組合收納櫃六層nmkd-556系列-2,2


In [49]:
df_select = df[df['pagePath'].str.contains("/merchandise/view/")]

In [50]:
df_select.sort_values(by = ['pageviews'],ascending=False)

,pagePath,pageviews
9676,/merchandise/view/129456,43209
9926,/merchandise/view/130620,30424
8379,/merchandise/view/125572,16039
9368,/merchandise/view/128165,8475
8616,/merchandise/view/125573,7538
...,...,...
7631,/merchandise/view/121779?bwp_lp=1&ug=4&iid=121...,0
11516,/merchandise/view/72944,0
7077,/merchandise/view/121342?bwp_lp=1&ug=2&iid=121...,0
8906,/merchandise/view/127168?bwp_lp=1&ug=2&iid=127...,0


In [51]:
df_select['pagePath'] = df_select['pagePath'].str.replace('/merchandise/view/','')

/Users/wangxiang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
df_select['pagePath'] = df_select['pagePath'].str.split('?',n = 1, expand = True)

/Users/wangxiang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
df_select

,pagePath,pageviews
4471,100031,4
4472,100049,16
4473,100061,4
4474,100292,4
4475,10052,4
...,...,...
12426,9983,16
12427,99857,4
12428,9986,4
12429,9989,12


In [54]:
df_select = df_select.reset_index(drop=True)

In [56]:
df_select = df_select.drop(index=[7959])

In [57]:
df_select

,pagePath,pageviews
0,100031,4
1,100049,16
2,100061,4
3,100292,4
4,10052,4
...,...,...
7954,99824,4
7955,9983,16
7956,99857,4
7957,9986,4


In [58]:
df_select = df_select.rename(columns= {'pagePath':'prod_id'})

In [59]:
df_select.to_csv('table3.csv')

In [62]:
cart = pd.read_csv('cart_item_07.csv')

In [63]:
sale = pd.read_csv('sale_item_07.csv')

In [64]:
df_sale = sale.merge(cart,on = 'prod_id',how = 'left')

In [66]:
df_select['prod_id'] = pd.to_numeric(df_select['prod_id'],errors='coerce')

In [67]:
df_sale = df_sale.merge(df_select,on = 'prod_id',how = 'left')

In [68]:
df_sale = df_sale.fillna(0)

In [69]:
df_sale = df_sale.rename(columns = {'prod_id':'商品ID','sum(quantity)':'加入購物車數量','sessions':'流量','pageviews':'PV'})

In [70]:
df_sale['ROAS'] = df_sale['營業額']/df_sale['PV']

In [72]:
df_sale['ROAS'] = df_sale['ROAS'].round(1)

In [73]:
df_sale.head()

,商品ID,商品名稱,訂單數,銷量,營業額,成本,加入購物車數量,PV,ROAS
0,398,【船井生醫 funcare】6X活力瑪卡王(60顆/盒),1,1,520,442,0.0,4.0,130.0
1,455,全罩3D立體透氣口罩100片送液滅菌100ml 1(100入-M號 9-11cm成年女性用 ),2,2,1198,553,1.0,60.0,20.0
2,503,【三得利suntory 】蜂王乳 芝麻明E 120錠/瓶 (單罐(120錠)),1,1,1775,1596,0.0,36.0,49.3
3,581,【Simply新普利】蜂王乳夜酵素EX 30錠/盒,1,1,899,854,3.0,63.0,14.3
4,629,【Dr’s Formula 】胺基酸清爽肌沐浴乳(雪皂香氛)800g - 3入組,1,1,580,522,0.0,0.0,inf


In [74]:
df_sale['總成本'] = df_sale['銷量'] * df_sale['成本']

In [75]:
df_sale['毛利率'] = (1 - (df_sale['總成本'] / df_sale['營業額'])) *100

In [76]:
df_sale = df_sale.replace([np.inf, -np.inf], np.nan)

In [77]:
df_sale = df_sale.fillna(0)

In [78]:
df_sale = df_sale.round(0)

In [79]:
df_sale[df_sale['ROAS'] > 5]

,商品ID,商品名稱,訂單數,銷量,營業額,成本,加入購物車數量,PV,ROAS,總成本,毛利率
0,398,【船井生醫 funcare】6X活力瑪卡王(60顆/盒),1,1,520,442,0.0,4.0,130.0,442,15.0
1,455,全罩3D立體透氣口罩100片送液滅菌100ml 1(100入-M號 9-11cm成年女性用 ),2,2,1198,553,1.0,60.0,20.0,1106,8.0
2,503,【三得利suntory 】蜂王乳 芝麻明E 120錠/瓶 (單罐(120錠)),1,1,1775,1596,0.0,36.0,49.0,1596,10.0
3,581,【Simply新普利】蜂王乳夜酵素EX 30錠/盒,1,1,899,854,3.0,63.0,14.0,854,5.0
6,1721,【福義軒】大包裝手工蛋捲家庭號3包組(留言備註自選口味(6口味共任選3包)),1,1,780,751,0.0,40.0,20.0,751,4.0
...,...,...,...,...,...,...,...,...,...,...,...
2673,132112,《團》Panasonic國際牌 奈米離子保濕美顏器 【EH-SA31-VP】,1,1,3380,3288,0.0,28.0,121.0,3288,3.0
2674,132121,《團》【AVEDA】隨行按摩梳 2入組,6,6,6594,1044,0.0,36.0,183.0,6264,5.0
2675,132135,《團》【Tommy Hilfiger】經典刺繡LOGO圓領短上衣 四色(深藍色 M),7,10,6990,650,10.0,108.0,65.0,6500,7.0
2676,132154,《團》【柳宗理】日本製不銹鋼食物夾,1,2,1180,550,0.0,44.0,27.0,1100,7.0


In [194]:
df_sale.to_csv('sale_report2.csv')

# 流量

In [287]:
sessionload = {
    'start_date': '2020-07-12',
    'end_date': '2020-07-25',
    'metrics':'ga:sessions, ga:pageviews,ga:uniquePageviews',
    'dimensions': 'ga:week,ga:channelGrouping',
    'max_results':100
}
df_session = query.run_query(service, '111816020', sessionload)

In [288]:
df_session

,week,channelGrouping,sessions,pageviews,uniquePageviews
0,29,(Other),3304,62738,21807
1,29,Direct,9966,122147,43135
2,29,Display,8333,74159,28232
3,29,Email,488,8415,2554
4,29,Organic Search,18425,182050,63257
5,29,Paid Search,10,11,10
6,29,Referral,2167,34799,11382
7,29,Social,2472,19287,6907
8,30,(Other),3379,62440,21150
9,30,Direct,10090,101722,36745


In [289]:
df_session.to_csv('session_2021071225.csv')

In [79]:
df_session_pivot = pd.pivot_table(df_session,index = ['channelGrouping','source'],values = ['sessions','pageviews'],
                           aggfunc = {
                               'sessions':'sum',
                               'pageviews':'sum'
                           })

In [80]:
df_session_pivot

pageviews  sessions
channelGrouping source                              
(Other)         BN                     134         8
                D                      332        13
                GDN                      1         1
                LP                     475         2
                Line                    16         2
...                                    ...       ...
Social          l.instagram.com         24         3
                lm.facebook.com         71         4
                m.facebook.com       47233      3546
                page.line.me           603        13
                plurk.com               44         3

[84 rows x 2 columns]

In [200]:
#ga['Prod_Id'] = ga['Prod_Id'].str.replace('/merchandise/view/','')
#ga['Prod_Id'] = ga['Prod_Id'].str.replace('/?flag=3','')

In [201]:
#ga['Prod_Id'] = ga['Prod_Id'].str.replace('?','')
#ga['Prod_Id'] = pd.to_numeric(ga['Prod_Id'],errors='coerce')

In [202]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 商品/收益

In [274]:
ecload = {
    'start_date': '2021-07-12',
    'end_date': '2021-07-18',
    'metrics': 'ga:pageValue',
    'dimensions': 'ga:pagePath,ga:channelGrouping'#ga:productName
}
ec = query.run_query(service, '111816020', ecload)

In [275]:
ec

,pagePath,channelGrouping,pageValue
0,/,(Other),612.402062
1,/,Direct,300.742933
2,/,Display,618.180887
3,/,Email,115.083969
4,/,Organic Search,184.306069
...,...,...,...
26200,/terms,Display,0.000000
26201,/terms,Email,0.000000
26202,/terms,Organic Search,676.428571
26203,/terms,Paid Search,0.000000


In [276]:
ec_select = ec[ec['pagePath'].str.contains("/merchandise/view/")]

In [277]:
ec_select['pageValue'] = ec_select['pageValue'].round(1)

/Users/wangxiang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [278]:
ec_select['pagePath'] = ec_select['pagePath'].str.replace('/merchandise/view/','')

/Users/wangxiang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [279]:
ec_select['pagePath'] = ec_select['pagePath'].str.split(pat="?",expand = True)

/Users/wangxiang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [280]:
ec_select.to_csv('table4.csv')